In [1]:
import numpy as np
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data.sampler import SubsetRandomSampler
import matplotlib.pyplot as plt
from torch.utils.data import ConcatDataset
from PIL import Image
import os
import torchvision.models as models
import time
import copy
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder

from dataloading import load_data


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

c:\Users\Lenovo\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [24]:

transform = transforms.Compose([ 
    transforms.Resize((227, 227)),  
    transforms.ToTensor()
])    

data_percentage = 50


train_loader, _, _ = load_data(data_dir = '../data/vegetable_images',
                           batch_size = 64,
                           data_type = "train",
                           noise_type = "None",
                           noise_percentage = 0,                           
                           transform = transform,                           
                           data_percentage=data_percentage)

valid_loader, _, _ = load_data(data_dir = '../data/vegetable_images',
                           batch_size = 64,
                           data_type = "validation",
                           noise_type = "None",
                           noise_percentage = 0,                           
                           transform = transform,                           
                           data_percentage=data_percentage)

test_loader, _, _ = load_data(data_dir = '../data/vegetable_images',
                           batch_size = 64,
                           data_type = "test",
                           noise_type = "None",
                           noise_percentage = 0,                           
                           transform = transform,                           
                           data_percentage=data_percentage)


path:  ../data/vegetable_images/train
needed_length: 7500, expected_length_per_class: 500
length of final dataset: 7500
path:  ../data/vegetable_images/validation
needed_length: 1500, expected_length_per_class: 100
length of final dataset: 1500
path:  ../data/vegetable_images/test
needed_length: 1500, expected_length_per_class: 100
length of final dataset: 1500


In [4]:

concat_dataset = ConcatDataset([train_loader.dataset, valid_loader.dataset, test_loader.dataset])

# Create a new data loader from the concatenated dataset
batch_size = 64  # Set your desired batch size
concat_loader = DataLoader(concat_dataset, batch_size=batch_size, shuffle=True)

In [10]:
images, _ = next(iter(concat_loader))
images.shape


torch.Size([64, 3, 227, 227])

In [13]:
flattened_tensor1 = images.view(images.size(0), images.size(1), -1)

print(flattened_tensor1.shape)
flattened_tensor1

torch.Size([64, 3, 51529])


tensor([[[0.3294, 0.3216, 0.3137,  ..., 0.1686, 0.2000, 0.1922],
         [0.3569, 0.3569, 0.3569,  ..., 0.3490, 0.3882, 0.3804],
         [0.4667, 0.4706, 0.4824,  ..., 0.2039, 0.2314, 0.2235]],

        [[0.0078, 0.0000, 0.0000,  ..., 0.0706, 0.0706, 0.0667],
         [0.0078, 0.0000, 0.0000,  ..., 0.1294, 0.1333, 0.1294],
         [0.0078, 0.0000, 0.0000,  ..., 0.0157, 0.0196, 0.0157]],

        [[0.2510, 0.4667, 0.4824,  ..., 0.7098, 0.7059, 0.7059],
         [0.2745, 0.4863, 0.4941,  ..., 0.7373, 0.7333, 0.7333],
         [0.1412, 0.3608, 0.3804,  ..., 0.7765, 0.7725, 0.7725]],

        ...,

        [[0.6275, 0.6314, 0.6392,  ..., 0.5922, 0.5961, 0.5961],
         [0.6235, 0.6275, 0.6353,  ..., 0.6000, 0.6000, 0.6000],
         [0.6431, 0.6471, 0.6549,  ..., 0.6588, 0.6706, 0.6706]],

        [[0.1373, 0.1569, 0.1804,  ..., 0.0039, 0.0431, 0.0314],
         [0.1451, 0.1647, 0.1961,  ..., 0.0000, 0.0235, 0.0039],
         [0.1961, 0.2157, 0.2431,  ..., 0.0000, 0.0078, 0.0000]],

 

In [14]:
images, _ = next(iter(concat_loader))
flattened_tensor2 = images.view(images.size(0), images.size(1), -1)

print(flattened_tensor2.shape)
flattened_tensor2

torch.Size([64, 3, 51529])


tensor([[[0.2392, 0.2510, 0.2941,  ..., 0.6314, 0.6353, 0.6353],
         [0.2549, 0.2784, 0.3294,  ..., 0.7333, 0.7373, 0.7373],
         [0.0549, 0.1059, 0.2118,  ..., 0.8314, 0.8353, 0.8353]],

        [[0.4431, 0.5176, 0.5765,  ..., 0.0157, 0.0627, 0.0863],
         [0.4196, 0.4863, 0.5373,  ..., 0.0627, 0.0980, 0.1216],
         [0.4353, 0.4784, 0.4980,  ..., 0.0000, 0.0235, 0.0471]],

        [[0.0314, 0.0510, 0.0353,  ..., 0.7137, 0.7137, 0.7098],
         [0.0549, 0.0745, 0.0706,  ..., 0.7569, 0.7569, 0.7529],
         [0.0000, 0.0000, 0.0000,  ..., 0.3882, 0.3961, 0.3922]],

        ...,

        [[0.3647, 0.3765, 0.4314,  ..., 0.1098, 0.1451, 0.2863],
         [0.3333, 0.3451, 0.4039,  ..., 0.0000, 0.0431, 0.1922],
         [0.3765, 0.3882, 0.4353,  ..., 0.0118, 0.0471, 0.1922]],

        [[0.3765, 0.2314, 0.1137,  ..., 0.1059, 0.1098, 0.0941],
         [0.4863, 0.3255, 0.1843,  ..., 0.2157, 0.2196, 0.2039],
         [0.1961, 0.0824, 0.0314,  ..., 0.0549, 0.0706, 0.0549]],

 

In [16]:
pixel_values = []
pixel_values.append(flattened_tensor1)
pixel_values.append(flattened_tensor2)
pixel_values



[tensor([[[0.3294, 0.3216, 0.3137,  ..., 0.1686, 0.2000, 0.1922],
          [0.3569, 0.3569, 0.3569,  ..., 0.3490, 0.3882, 0.3804],
          [0.4667, 0.4706, 0.4824,  ..., 0.2039, 0.2314, 0.2235]],
 
         [[0.0078, 0.0000, 0.0000,  ..., 0.0706, 0.0706, 0.0667],
          [0.0078, 0.0000, 0.0000,  ..., 0.1294, 0.1333, 0.1294],
          [0.0078, 0.0000, 0.0000,  ..., 0.0157, 0.0196, 0.0157]],
 
         [[0.2510, 0.4667, 0.4824,  ..., 0.7098, 0.7059, 0.7059],
          [0.2745, 0.4863, 0.4941,  ..., 0.7373, 0.7333, 0.7333],
          [0.1412, 0.3608, 0.3804,  ..., 0.7765, 0.7725, 0.7725]],
 
         ...,
 
         [[0.6275, 0.6314, 0.6392,  ..., 0.5922, 0.5961, 0.5961],
          [0.6235, 0.6275, 0.6353,  ..., 0.6000, 0.6000, 0.6000],
          [0.6431, 0.6471, 0.6549,  ..., 0.6588, 0.6706, 0.6706]],
 
         [[0.1373, 0.1569, 0.1804,  ..., 0.0039, 0.0431, 0.0314],
          [0.1451, 0.1647, 0.1961,  ..., 0.0000, 0.0235, 0.0039],
          [0.1961, 0.2157, 0.2431,  ..., 0.0000,

In [17]:
pixel_values = torch.cat(pixel_values, dim=0)
pixel_values.shape

torch.Size([128, 3, 51529])

In [20]:
mean = torch.mean(pixel_values, dim=0)
mean.shape

torch.Size([3, 51529])

In [21]:
mean_per_channel = mean.mean(dim=1)
mean_per_channel

tensor([0.4649, 0.4587, 0.3365])

In [25]:
total = []
for images, _ in train_loader:
    images, _ = next(iter(concat_loader))
    flattened_image = images.view(images.size(0), images.size(1), -1)
    total.append(flattened_image)
pixel_values = torch.cat(total, dim=0)
pixel_values.shape



torch.Size([7552, 3, 51529])

In [26]:
mean = torch.mean(pixel_values, dim=0)
mean.shape

torch.Size([3, 51529])

In [27]:
mean_per_channel = mean.mean(dim=1)
mean_per_channel

tensor([0.4680, 0.4647, 0.3441])

In [35]:
std = torch.std(pixel_values, dim=0)
std_per_channel = std.mean(dim=1)
std_per_channel

tensor([0.2315, 0.2262, 0.2399])

In [33]:
reshaped_pixel = pixel_values.view(1, 3, -1)
std = torch.std(reshaped_pixel, dim=0)
std

RuntimeError: [enforce fail at ..\c10\core\impl\alloc_cpu.cpp:72] data. DefaultCPUAllocator: not enough memory: you tried to allocate 4669764096 bytes.

In [34]:
mean = torch.mean(reshaped_pixel, dim=0)

RuntimeError: [enforce fail at ..\c10\core\impl\alloc_cpu.cpp:72] data. DefaultCPUAllocator: not enough memory: you tried to allocate 4669764096 bytes.